In [1]:
import albumentations as A
import numpy as np
import torch
import torchvision
from PIL import Image

In [2]:
def calculate_margin(probs):
    sorted_probs, _ = torch.sort(probs, dim=1, descending=True)
    return sorted_probs[:, 0] - sorted_probs[:, 1]

def output_transform(pred):
    final_output = []
    final_output.append(pred.get("scores").tolist())
    return final_output

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = torch.jit.load('model.pt').to(device)

transforms = A.load('transforms.json')
# image_path = '/path/to/your/image'
image_path = '/home/wattx/hasty/images/cats/10000000001.jpg' # TODO: remove this
image = np.array(Image.open(image_path))
image = transforms(image=image)['image']

In [5]:
# Convert to torch tensor
x = torch.from_numpy(image).to(device)
with torch.no_grad():
    # Convert to channels first, convert to float datatype
    x = x.permute(2, 0, 1).float()
    y = model(x)

    # Calculate active learning scores
    image_instance_scores = output_transform(y)
    # NOTE: image instance scores should be a list of class probabilities for every instance
    # It's not true for this example cause exported Hasty model is not supported that atm
    # e.g. for OD  with two classes and three predictions, it should look like 
    # [[0.2, 0.8], [0.1, 0.9], [0.45, 0.55]]
    #
    # probs = torch.Tensor([[0.2, 0.8], [0.1, 0.9], [0.45, 0.55]])
    # scores = calculate_margin(torch.as_tensor(probs))
    # print(torch.min(scores).item())
    # >  0.10000002384185791
    
    # For YoLO probability vectore should exists in results:
    # detector = DetectMultiBackend(weights)
    # result = detector.mode.predict({'image': im})
    
    scores = 0
    if len(image_instance_scores):
        scores = calculate_margin(torch.as_tensor(image_instance_scores))
        scores = torch.min(scores).item()
scores

0.04942387342453003

Note: The lower the score the less certain the model about this image. Usually we suggest to have a threshold < 0.2, as a candidate for annotation. If the score is 0, it means that model haven't detect anything, might be something to check why